# 🚗 자동차 센서 데이터 분석 실습 노트북
본 노트북에서는 Python을 활용하여 자동차의 센서 데이터를 불러오고, 전처리 및 분석하는 과정을 실습합니다.

In [ ]:
# 📦 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')

## 📁 데이터 불러오기

In [ ]:
import gdown
import zipfile
import os

file_id = "1kqut3OQ5T0UpccE2LdxSD5UaPbbb-1ja"
gdown.download(f"https://drive.google.com/uc?id={file_id}", quiet=False)

In [ ]:
zip_path = "/content/sensor_data.zip"

extract_path = "sensor_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"압축 해제 완료")

In [ ]:
# Google Colab 환경일 경우 아래 주석을 해제하고 파일 업로드
# from google.colab import files
# uploaded = files.upload()

speed_df = pd.read_csv('./sensor_data/speed.csv', parse_dates=['timestamp'])
accel_df = pd.read_csv('./sensor_data/accel.csv', parse_dates=['timestamp'])
gps_df = pd.read_csv('./sensor_data/gps.csv', parse_dates=['timestamp'])

## 🔗 데이터 병합 및 결측치 처리

In [ ]:
merged_df = pd.merge_asof(speed_df.sort_values('timestamp'),
                           accel_df.sort_values('timestamp'),
                           on='timestamp')
merged_df = pd.merge_asof(merged_df.sort_values('timestamp'),
                          gps_df.sort_values('timestamp'),
                          on='timestamp')

# 결측치 보간
merged_df.set_index('timestamp', inplace=True)
merged_df = merged_df.interpolate(method='time').dropna()
print(merged_df.head())

merged_df.reset_index(inplace=True)

## 📊 속도 시각화

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(merged_df['timestamp'], merged_df['speed_kmh'])
plt.title("Vehicle Speed Over Time")
plt.xlabel("Time")
plt.ylabel("Speed (km/h)")
plt.grid(True)
plt.show()

## ⚠️ 급가속/급감속 이벤트 탐지

In [ ]:
merged_df['delta_speed'] = merged_df['speed_kmh'].diff()
merged_df['delta_time'] = merged_df['timestamp'].diff().dt.total_seconds()
merged_df['acceleration'] = merged_df['delta_speed'] / merged_df['delta_time']

rapid_event_df = merged_df[(merged_df['acceleration'] > 2) | (merged_df['acceleration'] < -2)]
rapid_event_df[['timestamp', 'speed_kmh', 'acceleration']]

## 🗺️ GPS 경로 시각화

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(merged_df['longitude'], merged_df['latitude'], marker='o', markersize=3)
plt.title("Driving Path (GPS)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.axis('equal')
plt.grid(True)
plt.show()

## 📌 요약 통계

In [ ]:
print("평균 속도:", merged_df['speed_kmh'].mean())
print("최대 속도:", merged_df['speed_kmh'].max())
print("급가속 이벤트 수:", len(rapid_event_df))

##🎯 시계열 예측 (LSTM을 활용한 속도 예측)

- LSTM을 이용해 speed_kmh 시계열 데이터를 학습하고 향후 몇 초간의 속도를 예측

- 데이터 전처리, 시퀀스 생성, 모델 학습 및 예측까지 단계별로 진행

🔹 Step 1: 필요 라이브러리 설치 및 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

🔹 Step 2: 데이터 로딩 및 준비

In [ ]:
# CSV 파일 불러오기
df = pd.read_csv('./sensor_data/speed.csv', parse_dates=['timestamp'])

# 속도만 사용
speed = df[['speed_kmh']].values

# 정규화
scaler = MinMaxScaler()
speed_scaled = scaler.fit_transform(speed)

🔹 Step 3: 시계열 데이터 시퀀스 생성

In [ ]:
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

SEQ_LEN = 10  # 입력 시퀀스 길이
X, y = create_sequences(speed_scaled, SEQ_LEN)

# 학습/테스트 분리
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

🔹 Step 4: LSTM 모델 구성 및 학습

In [ ]:
model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

🔹 Step 5: 예측 및 시각화

In [ ]:
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

plt.figure(figsize=(12, 5))
plt.plot(y_test_inv, label='True Speed')
plt.plot(y_pred_inv, label='Predicted Speed')
plt.title("LSTM Speed Prediction")
plt.xlabel("Time Step")
plt.ylabel("Speed (km/h)")
plt.legend()
plt.grid()
plt.show()

##🚗 다변량 시계열 예측: 자동차 센서 데이터로 속도 예측


🔍 실습 목표

- 다중 센서 데이터를 활용한 다변량 시계열 예측

- 입력: 가속도(x, y, z), 위치(lat/lon), 과거 속도

- 출력: 미래 1초 후의 속도 예측

- 모델: LSTM 및 Transformer 기반 비교

📦 라이브러리 불러오기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

📁 데이터 불러오기 및 전처리

In [ ]:
# CSV 파일 로드
df = pd.read_csv('./sensor_data/merged_multisensor_data.csv', parse_dates=['timestamp'])
df = df.drop(columns=['timestamp'])  # 타임스탬프 제거 (LSTM에 직접 사용하지 않음)

# 스케일링
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

# 시퀀스 생성 함수
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length][0])  # speed_kmh 예측
    return np.array(X), np.array(y)

SEQ_LEN = 10
X, y = create_sequences(data_scaled, SEQ_LEN)

X.shape, y.shape

🧠 LSTM 모델 학습

In [ ]:
# 데이터 분할
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# LSTM 모델 정의
model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, X.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

📊 LSTM 예측 시각화

In [ ]:
# 예측
y_pred = model.predict(X_test)

# 역변환
speed_scaler = MinMaxScaler()
speed_scaler.min_, speed_scaler.scale_ = scaler.min_[0], scaler.scale_[0]
y_test_inv = y_test / speed_scaler.scale_ + speed_scaler.min_
y_pred_inv = y_pred[:, 0] / speed_scaler.scale_ + speed_scaler.min_

# 시각화
plt.figure(figsize=(12, 5))
plt.plot(y_test_inv, label='True Speed')
plt.plot(y_pred_inv, label='Predicted Speed')
plt.title("Multivariate LSTM Speed Prediction")
plt.xlabel("Time Step")
plt.ylabel("Speed (km/h)")
plt.legend()
plt.grid()
plt.show()

🔁 Transformer 기반 예측 모델

In [ ]:
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout, Input, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Transformer 블록 정의
def build_transformer_model(input_shape):
    inputs = Input(shape=input_shape)
    x = LayerNormalization()(inputs)
    x = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x = Dropout(0.2)(x)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(1)(x)
    return Model(inputs, outputs)

# 모델 생성 및 학습
transformer_model = build_transformer_model((SEQ_LEN, X.shape[2]))
transformer_model.compile(optimizer=Adam(1e-4), loss='mse')
transformer_model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

📈 Transformer 예측 결과 시각화

In [ ]:
y_pred_tf = transformer_model.predict(X_test)
y_pred_tf_inv = y_pred_tf[:, 0] / speed_scaler.scale_ + speed_scaler.min_

plt.figure(figsize=(12, 5))
plt.plot(y_test_inv, label='True Speed')
plt.plot(y_pred_tf_inv, label='Transformer Prediction')
plt.title("Transformer Speed Prediction")
plt.xlabel("Time Step")
plt.ylabel("Speed (km/h)")
plt.legend()
plt.grid()
plt.show()

##🚗 차량 센서 데이터 이상 탐지 실습 (Autoencoder 기반)

이 실습에서는 Autoencoder 딥러닝 모델을 활용하여 차량 속도 데이터를 기반으로 이상 가속도 구간을 자동으로 탐지합니다.

📦 라이브러리 불러오기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')

📁 데이터 불러오기

In [ ]:
# anomaly_speed_data.csv 파일 로드
df = pd.read_csv('./sensor_data/anomaly_speed_data.csv', parse_dates=['timestamp'])
df.head()

📊 속도 및 가속도 시각화

In [ ]:
# 속도 시각화
plt.figure(figsize=(12, 4))
plt.plot(df['timestamp'], df['speed_kmh'], label='Speed (km/h)')
plt.title("Vehicle Speed Over Time")
plt.xlabel("Time")
plt.ylabel("Speed (km/h)")
plt.grid(True)
plt.legend()
plt.show()

# 가속도 시각화
plt.figure(figsize=(12, 4))
plt.plot(df['timestamp'], df['acceleration'], label='Acceleration (m/s²)', color='orange')
plt.title("Acceleration Over Time")
plt.xlabel("Time")
plt.ylabel("Acceleration (m/s²)")
plt.grid(True)
plt.legend()
plt.show()

🧠 Autoencoder 모델 정의 및 학습

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# 데이터 스케일링
scaler = MinMaxScaler()
X = scaler.fit_transform(df[['speed_kmh', 'acceleration']].fillna(0))

# Autoencoder 모델 정의
input_dim = X.shape[1]
inputs = Input(shape=(input_dim,))
encoded = Dense(8, activation='relu')(inputs)
encoded = Dense(4, activation='relu')(encoded)
decoded = Dense(8, activation='relu')(encoded)
decoded = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer=Adam(1e-3), loss='mse')
autoencoder.fit(X, X, epochs=50, batch_size=16, shuffle=True, validation_split=0.1)

📌 이상 탐지: 재구성 오류 기반

In [ ]:
# 예측 및 재구성 오차 계산
X_pred = autoencoder.predict(X)
recon_error = np.mean(np.square(X - X_pred), axis=1)
df['reconstruction_error'] = recon_error

# 임계값: 평균 + 3*표준편차
threshold = recon_error.mean() + 3 * recon_error.std()
df['anomaly'] = df['reconstruction_error'] > threshold

# 이상 탐지된 항목 확인
anomalies = df[df['anomaly'] == True]
print(f"총 {len(anomalies)}건의 이상이 탐지되었습니다.")
display(anomalies[['timestamp', 'speed_kmh', 'acceleration', 'reconstruction_error']])

📈 재구성 오차 및 이상치 시각화

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(df['timestamp'], df['reconstruction_error'], label='Reconstruction Error')
plt.axhline(threshold, color='red', linestyle='--', label='Threshold')
plt.title("Autoencoder Reconstruction Error Over Time")
plt.xlabel("Time")
plt.ylabel("Error")
plt.legend()
plt.grid(True)
plt.show()

🚨 이상 속도 구간 시각화

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df['timestamp'], df['speed_kmh'], label='Speed')
plt.scatter(anomalies['timestamp'], anomalies['speed_kmh'], color='red', label='Detected Anomaly', zorder=5)
plt.title("Detected Speed Anomalies using Autoencoder")
plt.xlabel("Time")
plt.ylabel("Speed (km/h)")
plt.legend()
plt.grid(True)
plt.show()